# 🎯 QML Pattern Visualization Hub

**Zero-Config Professional Pattern Analysis System**

---

## 📋 Overview

This notebook provides interactive visualization of QML patterns with:
- **Auto-loading** from Master Data Store (no manual OHLCV config needed)
- Premium TradingView-style charts
- Interactive pattern selection
- Trade zone visualization (entry, stop-loss, take-profit)

## 🚀 Quick Start

1. **Set `PATTERNS_CSV`** path in Cell 1 (only input required!)
2. **Run All Cells** - data is auto-loaded from Master Data Store

---

## ⚙️ Step 1: Pattern File Configuration

**Only input required** - OHLCV data is auto-loaded from Master Data Store

In [1]:
# =============================================================================
# 📁 SINGLE INPUT CONFIGURATION
# =============================================================================
# Point this to any pattern CSV from your experiments folder

from pathlib import Path

PATTERNS_CSV = Path('./experiments/exp_01_rolling_v1/patterns.csv')

# Optional: Override to use local OHLCV instead of Master Data Store
USE_MASTER_STORE = True  # Set to False to use experiment's local ohlcv.csv

print(f"✅ Configuration set")
print(f"   Patterns: {PATTERNS_CSV}")
print(f"   Data Source: {'Master Data Store' if USE_MASTER_STORE else 'Local OHLCV'}")

✅ Configuration set
   Patterns: experiments/exp_01_rolling_v1/patterns.csv
   Data Source: Master Data Store


## 📦 Step 2: Auto-Load Data

In [2]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================

import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Enable inline plotting
%matplotlib inline

print("✅ Libraries loaded")

# =============================================================================
# LOAD PATTERN DATA
# =============================================================================

patterns_df = pd.read_csv(PATTERNS_CSV, parse_dates=[
    'TS_Date', 'P1_Date', 'P2_Date', 'P3_Date', 'P4_Date', 'P5_Date'
])

print(f"✅ Loaded {len(patterns_df)} patterns")
print(f"   Date range: {patterns_df['P5_Date'].min().strftime('%Y-%m-%d')} to {patterns_df['P5_Date'].max().strftime('%Y-%m-%d')}")
print(f"   Validity range: {patterns_df['validity_score'].min():.3f} - {patterns_df['validity_score'].max():.3f}")

# =============================================================================
# AUTO-DETECT TIMEFRAME FROM PATTERN DATA
# =============================================================================

def infer_timeframe(df):
    """Infer timeframe from pattern point spacing."""
    deltas = []
    for _, row in df.head(10).iterrows():  # Sample first 10 patterns
        if pd.notna(row.get('P1_Date')) and pd.notna(row.get('P2_Date')):
            delta = (row['P2_Date'] - row['P1_Date']).total_seconds()
            if delta > 0:
                deltas.append(delta)
    
    if not deltas:
        return '1h'  # Default
    
    median_delta = sorted(deltas)[len(deltas) // 2]
    # If median gap > 2.5 hours between points, likely 4h data
    return '4h' if median_delta > 9000 else '1h'

detected_timeframe = infer_timeframe(patterns_df)
print(f"   Detected timeframe: {detected_timeframe}")

# =============================================================================
# AUTO-LOAD OHLCV DATA
# =============================================================================

if USE_MASTER_STORE:
    # Load from Master Data Store
    MASTER_DATA_DIR = Path('./data/processed/BTC')
    master_file = MASTER_DATA_DIR / f"{detected_timeframe}_master.parquet"
    
    if master_file.exists():
        ohlcv_df = pd.read_parquet(master_file)
        ohlcv_df = ohlcv_df.set_index('time')
        print(f"✅ Loaded Master Data Store: {master_file.name}")
    else:
        print(f"⚠️ Master data not found: {master_file}")
        print(f"   Falling back to local OHLCV...")
        USE_MASTER_STORE = False

if not USE_MASTER_STORE:
    # Fallback: Load local OHLCV from experiment folder
    local_ohlcv = PATTERNS_CSV.parent / 'ohlcv.csv'
    if local_ohlcv.exists():
        ohlcv_df = pd.read_csv(local_ohlcv, parse_dates=['time'])
        ohlcv_df = ohlcv_df.set_index('time')
        print(f"✅ Loaded local OHLCV: {local_ohlcv}")
    else:
        raise FileNotFoundError(f"No OHLCV data found. Build Master Store: python -m src.data_engine")

print(f"   OHLCV rows: {len(ohlcv_df)}")
print(f"   OHLCV range: {ohlcv_df.index.min().strftime('%Y-%m-%d')} to {ohlcv_df.index.max().strftime('%Y-%m-%d')}")

# Display summary statistics
print(f"\n📊 Pattern Distribution:")
print(f"   Bullish: {len(patterns_df[patterns_df['pattern_type'] == 'bullish_qml'])} patterns")
print(f"   Bearish: {len(patterns_df[patterns_df['pattern_type'] == 'bearish_qml'])} patterns")

✅ Libraries loaded
✅ Loaded 40 patterns
   Date range: 2023-01-19 to 2023-12-19
   Validity range: 0.580 - 0.898
   Detected timeframe: 4h
✅ Loaded Master Data Store: 4h_master.parquet
   OHLCV rows: 10950
   OHLCV range: 2021-01-04 to 2026-01-03

📊 Pattern Distribution:
   Bullish: 22 patterns
   Bearish: 18 patterns


## 🎨 Step 3: Load Visualization Engine

In [3]:
# =============================================================================
# PREMIUM PATTERN VISUALIZATION FUNCTION
# =============================================================================

def plot_pattern(pattern_id):
    """
    🎨 TradingView-quality QML pattern visualization
    
    Features:
    - 60+ bars context (before/after pattern)
    - Color-coded trade zones (green=TP, red=SL)
    - Pattern structure overlay (TS→P1→P2→P3→P4→P5)
    - Entry marker and risk/reward metrics
    """
    
    if pattern_id < 0 or pattern_id >= len(patterns_df):
        print(f"❌ Invalid pattern_id. Must be 0-{len(patterns_df)-1}")
        return
    
    pattern = patterns_df.iloc[pattern_id]
    is_bullish = 'bullish' in pattern['pattern_type']
    
    # Extract pattern coordinates
    points = {
        'TS': (pd.Timestamp(pattern['TS_Date']), float(pattern['TS_Price'])),
        'P1': (pd.Timestamp(pattern['P1_Date']), float(pattern['P1_Price'])),
        'P2': (pd.Timestamp(pattern['P2_Date']), float(pattern['P2_Price'])),
        'P3': (pd.Timestamp(pattern['P3_Date']), float(pattern['P3_Price'])),
        'P4': (pd.Timestamp(pattern['P4_Date']), float(pattern['P4_Price'])),
        'P5': (pd.Timestamp(pattern['P5_Date']), float(pattern['P5_Price'])),
    }
    
    entry = float(pattern['entry_price'])
    sl = float(pattern['stop_loss'])
    tp = float(pattern['take_profit'])
    
    # Slice data with padding
    ts_idx = ohlcv_df.index.get_indexer([points['TS'][0]], method='nearest')[0]
    p5_idx = ohlcv_df.index.get_indexer([points['P5'][0]], method='nearest')[0]
    
    start_idx = max(0, ts_idx - 60)
    end_idx = min(len(ohlcv_df) - 1, p5_idx + 60)
    
    chart_df = ohlcv_df.iloc[start_idx:end_idx + 1].copy()
    
    if len(chart_df) < 30:
        print(f"⚠️ Warning: Limited data ({len(chart_df)} bars)")
    
    # Calculate dynamic label offset
    price_range = chart_df['High'].max() - chart_df['Low'].min()
    label_offset = price_range * 0.04
    
    # Build pattern structure lines
    trend_line = [(points['TS'][0], points['TS'][1]), (points['P1'][0], points['P1'][1])]
    qml_line = [(points['P1'][0], points['P1'][1]), (points['P2'][0], points['P2'][1]),
                (points['P3'][0], points['P3'][1]), (points['P4'][0], points['P4'][1]),
                (points['P5'][0], points['P5'][1])]
    
    alines_spec = dict(
        alines=[trend_line, qml_line],
        colors=['#808080', '#2962ff'],
        linewidths=[1.2, 2.5],
        linestyle=['-', '-'],
    )
    
    # Create chart
    mc = mpf.make_marketcolors(up='#26a69a', down='#ef5350', edge='inherit', wick='inherit')
    style = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc, gridstyle='-',
                                gridcolor='#e8e8e8', facecolor='white', figcolor='white')
    
    title = f"QML {pattern['pattern_type'].upper()} | Pattern #{pattern_id} | Validity: {pattern['validity_score']:.2f}"
    
    fig, axes = mpf.plot(chart_df, type='candle', style=style, title=title, ylabel='Price (USDT)',
                         volume=False, figsize=(18, 10), alines=alines_spec, returnfig=True,
                         tight_layout=True, scale_padding={'left': 0.1, 'right': 0.1, 'top': 0.3, 'bottom': 0.2})
    
    ax = axes[0]
    
    # Helper function for x-axis positioning
    def get_x_pos(timestamp):
        if timestamp in chart_df.index:
            return list(chart_df.index).index(timestamp)
        else:
            return chart_df.index.get_indexer([timestamp], method='nearest')[0]
    
    p5_x = get_x_pos(points['P5'][0])
    x_max = len(chart_df) - 1
    box_width = x_max - p5_x
    
    # Draw trade zones
    if is_bullish:
        sl_rect = patches.Rectangle((p5_x, sl), box_width, entry - sl, linewidth=0,
                                     facecolor='#ef5350', alpha=0.25, zorder=0)
        tp_rect = patches.Rectangle((p5_x, entry), box_width, tp - entry, linewidth=0,
                                     facecolor='#26a69a', alpha=0.25, zorder=0)
        ax.add_patch(sl_rect)
        ax.add_patch(tp_rect)
    else:
        sl_rect = patches.Rectangle((p5_x, entry), box_width, sl - entry, linewidth=0,
                                     facecolor='#ef5350', alpha=0.25, zorder=0)
        tp_rect = patches.Rectangle((p5_x, tp), box_width, entry - tp, linewidth=0,
                                     facecolor='#26a69a', alpha=0.25, zorder=0)
        ax.add_patch(sl_rect)
        ax.add_patch(tp_rect)
    
    # Entry marker
    ax.plot(p5_x, entry, marker='^' if is_bullish else 'v', markersize=14,
            color='#2962ff', markeredgecolor='white', markeredgewidth=1.5, zorder=15)
    
    # Labels
    label_style = dict(fontsize=10, fontweight='bold', color='#333333', ha='center',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                                edgecolor='#888888', alpha=0.92, linewidth=0.8), zorder=20)
    
    labels_config = {
        'P2': ('LS', 'above'),
        'P3': ('H', 'above' if not is_bullish else 'below'),
        'P4': ('LL', 'below' if is_bullish else 'above'),
        'P5': ('RS', 'above'),
    }
    
    for pt_name, (label_text, position) in labels_config.items():
        pt_time, pt_price = points[pt_name]
        x_pos = get_x_pos(pt_time)
        y_pos = pt_price + label_offset if position == 'above' else pt_price - label_offset
        va = 'bottom' if position == 'above' else 'top'
        ax.annotate(label_text, xy=(x_pos, pt_price), xytext=(x_pos, y_pos), va=va, **label_style)
    
    # Info box
    outcome = "BULLISH ▲" if is_bullish else "BEARISH ▼"
    risk = abs(entry - sl)
    reward = abs(tp - entry)
    rr = reward / risk if risk > 0 else 0
    
    info_text = f"{outcome}\nEntry: ${entry:,.2f}\nSL: ${sl:,.2f}\nTP: ${tp:,.2f}\nR:R = 1:{rr:.1f}"
    ax.text(0.98, 0.97, info_text, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', horizontalalignment='right', family='monospace',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='#cccccc', alpha=0.95, linewidth=1))
    
    # Horizontal lines
    ax.axhline(y=entry, color='#2962ff', linestyle='--', linewidth=1, alpha=0.7)
    ax.axhline(y=sl, color='#ef5350', linestyle='-', linewidth=1, alpha=0.5)
    ax.axhline(y=tp, color='#26a69a', linestyle='-', linewidth=1, alpha=0.5)
    
    plt.show()
    
    # Console output
    print(f"\n{'='*70}")
    print(f"📊 Pattern #{pattern_id}: {pattern['pattern_type'].upper()}")
    print(f"   Validity: {pattern['validity_score']:.3f} | Entry: ${entry:,.2f} | SL: ${sl:,.2f} | TP: ${tp:,.2f}")
    print(f"   Risk:Reward = 1:{rr:.2f} | Bars: {len(chart_df)}")
    print(f"{'='*70}")


print("✅ Visualization engine ready")

✅ Visualization engine ready


## 🎛️ Step 4: Interactive Pattern Selector

**Select and visualize any pattern from the dropdown**

In [ ]:
# =============================================================================
# INTERACTIVE PATTERN SELECTOR
# =============================================================================

# Create dropdown options sorted by validity score
dropdown_options = []
for i, row in patterns_df.iterrows():
    date_str = row['P5_Date'].strftime('%Y-%m-%d')
    label = f"#{i:02d} | {row['pattern_type']:12} | {date_str} | Validity: {row['validity_score']:.3f}"
    dropdown_options.append((label, i))

dropdown_options.sort(key=lambda x: patterns_df.iloc[x[1]]['validity_score'], reverse=True)

# Create widgets
pattern_dropdown = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[0][1],
    description='Pattern:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='700px')
)

plot_button = widgets.Button(
    description='📊 Visualize',
    button_style='success',
    layout=widgets.Layout(width='120px'),
    tooltip='Click to plot selected pattern'
)

output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output(wait=True)
        plot_pattern(pattern_dropdown.value)

plot_button.on_click(on_button_click)

# Display interface
print("🎛️ Interactive Selector Active")
print(f"   Total Patterns: {len(patterns_df)}")
print(f"   Sorted by: Validity Score (highest first)\n")

display(widgets.HBox([pattern_dropdown, plot_button]))
display(output)

# Auto-plot highest validity pattern
with output:
    plot_pattern(dropdown_options[0][1])

🎛️ Interactive Selector Active
   Total Patterns: 40
   Sorted by: Validity Score (highest first)



Output()

---

## 🔧 Advanced: Manual Pattern Access

For programmatic access or batch processing

In [5]:
# =============================================================================
# QUICK PATTERN LOOKUP
# =============================================================================

# View top patterns by validity
def show_top_patterns(n=10):
    """Display top N patterns sorted by validity score"""
    print(f"\n🏆 Top {n} Patterns by Validity Score:")
    print(f"{'ID':<5} {'Type':<15} {'Date':<12} {'Validity':<10} {'Entry':<12} {'R:R':<8}")
    print("-" * 75)
    
    top = patterns_df.nlargest(n, 'validity_score')
    for i, row in top.iterrows():
        rr = abs(row['take_profit'] - row['entry_price']) / abs(row['entry_price'] - row['stop_loss'])
        date_str = row['P5_Date'].strftime('%Y-%m-%d')
        print(f"{i:<5} {row['pattern_type']:<15} {date_str:<12} {row['validity_score']:<10.3f} "
              f"${row['entry_price']:<11,.2f} 1:{rr:.1f}")

# Quick manual plot function
def quick_plot(pattern_id):
    """Quickly plot a pattern by ID"""
    plot_pattern(pattern_id)

# Show top patterns
show_top_patterns(10)

print("\n💡 Usage Examples:")
print("   quick_plot(8)              # Plot pattern #8")
print("   show_top_patterns(15)      # Show top 15 patterns")
print("   patterns_df.head()         # View raw data")


🏆 Top 10 Patterns by Validity Score:
ID    Type            Date         Validity   Entry        R:R     
---------------------------------------------------------------------------
8     bullish_qml     2023-03-16   0.898      $24,509.85   1:1.0
13    bearish_qml     2023-04-17   0.880      $30,270.60   1:1.0
15    bearish_qml     2023-05-01   0.790      $29,176.15   1:1.0
24    bearish_qml     2023-06-14   0.790      $25,780.30   1:1.0
38    bullish_qml     2023-12-08   0.758      $43,705.38   1:1.0
5     bearish_qml     2023-02-17   0.746      $24,552.27   1:1.0
12    bullish_qml     2023-04-14   0.733      $30,333.57   1:1.0
9     bearish_qml     2023-03-24   0.725      $27,439.62   1:1.0
21    bearish_qml     2023-05-16   0.722      $27,357.31   1:1.0
29    bearish_qml     2023-07-07   0.701      $30,305.78   1:1.0

💡 Usage Examples:
   quick_plot(8)              # Plot pattern #8
   show_top_patterns(15)      # Show top 15 patterns
   patterns_df.head()         # View raw data


---

## 📚 Data Sources

### Master Data Store
Pre-processed OHLCV data with ATR indicators, located at:
- `data/processed/BTC/1h_master.parquet`
- `data/processed/BTC/4h_master.parquet`

Build with: `python -m src.data_engine`

### Experiment Files
Pattern CSVs from experiments folder:
- `experiments/exp_01_rolling_v1/patterns.csv`
- `experiments/exp_02_atr_v2/patterns.csv`

### Required Pattern Columns
- `pattern_id`, `pattern_type`, `validity_score`
- `entry_price`, `stop_loss`, `take_profit`
- `TS_Date`, `TS_Price` through `P5_Date`, `P5_Price`